## Probability Distributions 
### Professors Warner and Horton

In this notebook, we will introduce some of the more common discrete and continuous probability distributions. These are commonly used to model random events.

### Discrete Distributions

Remember that a random variable is a mapping from the sample space to a real number, $X:S \rightarrow \mathbb{R}$. The *probability mass function* (pmf) is a function that maps each outcome of the random variable to a real number between 0 and 1, $f: \mathbb{R} \rightarrow [0,1]$, such that for all $x \in \mathbb{R}$, 

$$f(x)=P(X=x)$$

Another useful idea is that of the *cumulative distribution function* (cdf) which is:

$$ F_{X}(x) = P(X \leq x)$$


The pmf can be written as an explicit function, a table, a plot, or words. We will focus on the first. Most problems take a wording and convert it to a table of function.

##### Example  

A team is playing a two game series against another team. The probability of winning the first game is 0.52 and the probability of winning the second, your star player has a planned absence, is 0.43. Let $X$ be the number of games won in the two played. 

1) Determine the probability mass function as a table.  
2) Find the cumulative distribution function as a table.  
3) Find the probability of winning at least one game.  

In [7]:
# Run this code to import packages
import datascience as ds
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

The random variable $X$ the number of games won in two played. It can take the values ${0,1,2}$. The sample space is ${LL,LW,WL,WW}$ which we will map to the random variable. Thus $LL \rightarrow 0$, $LW \rightarrow 1$,  $WL \rightarrow 1$, and  $WW \rightarrow 2$. The pmf is:

1) Find the probability mass function.

In [8]:
w1=.52
w2=.43
pmf_table=ds.Table().with_columns(["Number of Wins (x)",np.arange(3)])
probs= np.array([(1-w1)*(1-w2),(1-w1)*w2+w1*(1-w2),w1*w2])
pmf_table=pmf_table.with_columns(["Probability P(X=x)",probs])
pmf_table

Number of Wins (x),Probability P(X=x)
0,0.2736
1,0.5028
2,0.2236


2) Find the cumulative probability distribution.

In [9]:
def cdf(table):
    '''Given a two column table with values in first column and probability mass in the second,
    return a table of the cdf
    table - Two columns will realizations of the random variable in first and probability in second'''
    temp=table.select(0).with_columns("Cumulative Probability",np.cumsum(table[1]))
    return temp

In [10]:
cdf(pmf_table)

Number of Wins (x),Cumulative Probability
0,0.2736
1,0.7764
2,1


3) Find the probability of winning at least one game.

In [11]:
prob_winning_0 = pmf_table.column(1).item(2)
prob_at_least_one = 1-prob_winning_0
prob_at_least_one

0.7764

#### Moments

Random variables can be summarized with expected values. In general, let $X$ be a discrete random variable with pmf $f(x)$. Then $$E(g(X)) = \sum_{x}g(x)f(x)$$

The mean is $$E(X) = \mu = \sum_{x}xf(x),$$ also called the first moment about the origin, and variance is 
$$E((X- \mu)^{2})  = V(X) = \sigma ^{2} =\sum_{x}(x - \mu)^{2}f(x), $$ known as the second moment about the mean.

For our problem above, find the mean and variance.

In [12]:
pmf_table

Number of Wins (x),Probability P(X=x)
0,0.2736
1,0.5028
2,0.2236


Mean: 

In [13]:
np.sum(pmf_table.column(1)*pmf_table.column(0))

0.95

Variance:

In [14]:
total = 0
for i in pmf_table.column(0):
    total += (i-0.95)**2*pmf_table.column(1).item(i)
print(total)

0.49470000000000003


### Named Discrete Distributions

Helpfully, random variables often take on familiar "named" distributions. Such distributions occur often enough that we know the generic form of the pmf, expected value and variance. In these cases, we can avoid deriving the pmf like we did in 1) above. Below, we introduce many common "named" discrete distributions.

#### Binomial  

If each trial of an experiment has only two outcomes, arbitrarily named success and failure, where the probability of success of each trial is constant and independent of the prior trials and the number of trials is fixed in advance, then we have a binomial random variable.  
The general form of the random variable is $X$ is the number of successes in $n$ trials, where $p$ is the probability of success in a single trial.

The closed form solution to the pmf is $$f(x;n,p) = {n\choose x}p^{x}(1-p)^{n-x}$$

Write a function to find the pmf of a binomial and then use it to make a second cdf function. The following function will help.

In [15]:
from math import factorial

def choose(n, c):
    '''Number of ways to choose c items from a list of n items.'''
    return factorial(n) // (factorial(n - c) * factorial(c))

In [16]:
def my_binom(x,n,p):
    '''pmf of binomial r.v.; returns the probability of x successes out of n trials 
    where each trial has probability p of success'''
    return choose(n,x)*p**x*(1-p)**(n-x)

In [17]:
my_binom(1,2,.25)

0.375

In [18]:
def binom_table(n,p):
    '''Create the pmf table for a binomial'''
    pmf_table=ds.Table().with_columns("Number of Successes (x)",np.arange(n+1))
    pmf_table=pmf_table.with_columns("Probability P(X=x)",pmf_table.apply(lambda x:my_binom(x,n,p),0))
    return pmf_table

In [19]:
binom_table(4,.25)

Number of Successes (x),Probability P(X=x)
0,0.316406
1,0.421875
2,0.210938
3,0.046875
4,0.00390625


In [20]:
cdf(binom_table(4,.25))

Number of Successes (x),Cumulative Probability
0,0.316406
1,0.738281
2,0.949219
3,0.996094
4,1


##### Checking results

The scipy package has common "named" distributions built into its stats module. See the [link](https://docs.scipy.org/doc/scipy/reference/stats.html) to the package.

In [21]:
stats.binom.pmf(2,4,.25)

0.21093750000000006

In [22]:
stats.binom.cdf(2,4,.25)

0.94921875

The mean and variance of a binomial are $$E(X) = np$$ and $$V(X) = np(1-p)$$

##### Randomization

The scipy package also allows you to obtain random realizations from a random variable with a named distribution. 

In [23]:
rands=stats.binom.rvs(4,0.25,size=1000)
rands[:10]

array([0, 2, 3, 0, 3, 2, 0, 2, 1, 1])

Let's look at the mean and variance of our random sample and compare with the mean and variance above.

In [24]:
#Mean of random sample
print(rands.mean())
#Population mean
print(4*0.25)
#Var of random sample
print(rands.var())
#Population variance
print(4*0.25*0.75)

0.988
1.0
0.737856
0.75


##### Example 

Suppose you toss a fair die 12 times. Let $X$ be the number of rolls that resulted in 1 or 2.

1) Find the pmf of $X$.

2) Find the mean and variance of $X$.

3) Find the probability that $X$ is in the interval $[3,7]$. 

In [25]:
def binom_dice_table(n,p):
    '''Create the pmf table for a binomial'''
    pmf_dice_table=ds.Table().with_columns("Number of Successes (x)",np.arange(n+1))
    pmf_dice_table=pmf_dice_table.with_columns("Probability P(X=x)",pmf_dice_table.apply(lambda x:my_binom(x,n,p),0))
    return pmf_dice_table

In [26]:
binom_dice_table(12,(1/3))

Number of Successes (x),Probability P(X=x)
0,0.00770735
1,0.0462441
2,0.127171
3,0.211952
4,0.238446
5,0.190757
6,0.111275
7,0.0476892
8,0.0149029
9,0.00331175


In [27]:
dice_mean = 12 * (1/3)
print(dice_mean)
dice_variance = dice_mean*(1-(1/3))
print(dice_variance)

4.0
2.666666666666667


In [28]:
three_seven_table = binom_dice_table(12,(1/3)).take(np.arange(3,8))
prob_3_through_7 = cdf(three_seven_table).column(1).take(4)
prob_3_through_7

0.800118921949944

#### Negative Binomial  

Similar to binomial except that the failures, thus the trials, are random and the successes are fixed. Notice that the last trial has to be a success.  
The general form of the random variable is $X$ number of failures until the r<sup>th</sup> success.  
The geometric distribution is a special case where r is 1.
Since the trials are not fixed, $x = 0, 1, 2, 3, ...$

The closed form solution to the pmf is $$f(x;r,p) = {x+r-1\choose x}p^{r}(1-p)^{x}$$

The mean and variance of a negative binomial are $$E(X) = \frac{r}{p}-r$$ and $$V(X) = \frac{r(1-p)}{p^2}$$

##### Example

Suppose I am a 60% free throw shooter (the probability of making a free throw is 0.6, and all my free throws are independent with identical chance of success). I would like to make 5 free throws. Let $X$ be the number of misses before I make 5 free throws. 

1) Find the mean and variance of $X$.

2) Find the probability I only need 5 attempts to make 5 free throws.

3) Find the probability I need at least 10 attempts to make 5 free throws. 

In [29]:
mean_free_throw = (5/.6)-5
print(mean_free_throw)
variance_free_throw = (5*(1 - .6))/(.6**2)
print(variance_free_throw)

3.333333333333334
5.555555555555555


In [30]:
def neg_binom_pmf(x,r, p):
    n = x+r-1
    c = x
    return choose(n,c)*(p**r)*((1-p)**x)

In [31]:
neg_binom_pmf(5,5,.6)

0.10032906240000002

In [32]:
def neg_binom_ft_table(x,r,p):
    '''Create the pmf table for a negative binomial'''
    pmf_ft_table=ds.Table().with_columns("Number of Failures (x)",np.arange(x+1))

    pmf_ft_table=pmf_ft_table.with_columns("Probability P(X=x)",pmf_ft_table.apply(lambda x:neg_binom_pmf(x,r,p),0))
    
    return pmf_ft_table

In [33]:
five_in_five = neg_binom_ft_table(5,5,0.6).take(0).column(1)
five_in_five

array([0.07776])

In [34]:
at_least_ten = 1 - sum(neg_binom_ft_table(10,5,0.6).take(np.arange(0,5)).column(1))
at_least_ten

0.2665676800000001

#### Poisson

The Poisson distribution is common for modeling count or arrival data. It is common to model arrivals using the Poisson process. In this process, $X$, the number of arrivals in a specified amount of time, follows the Poisson distribution with parameter $\lambda$, the average number of arrivals in that specified amount of time. For $x = 0,1,2,3,...$, the closed form solution to the pmf is
$$
f(x;\lambda)=\frac{e^{-\lambda}\lambda^x}{x!}
$$

The mean and variance are $$E(X)=V(X)=\lambda$$

##### Example

Suppose fleet vehicles arrive to a maintenance garage at an average rate of 4 per day. Let's model these arrivals using the Poisson process. Let $X$ be the number of vehicles that arrive in a five day period. 

1) What is the distribution (with parameter) of $X$? 

2) Find the probability that fewer than 10 vehicles arrive in a five day period. 

3) Find the probability that at least 18 vehicles arrive in a five day period. 

1.) distribution: x ~ Pois(20)


In [35]:
import math
def poisson(x,l):
    dx = float(x)
    dl = float(l)
    ret = math.e**(-1 * dl) * dl**dx /factorial(dx)
    return ret

In [36]:
def poisson_table(x,l):
    '''Create the pmf table for a poisson'''
    poisson_table=ds.Table().with_columns("Number of Arrivals (x)",np.arange(x+1))
    
    poisson_table=poisson_table.with_columns("Probability P(X=x)", poisson_table.apply(lambda y:poisson(y,l),0))
    
    return poisson_table

In [37]:
poisson_table(5,4).show()

Number of Arrivals (x),Probability P(X=x)
0,0.0183156
1,0.0732626
2,0.146525
3,0.195367
4,0.195367
5,0.156293


In [38]:
fewer_than_10 = sum(poisson_table(10,20).column(1).take(np.arange(0,10)))
fewer_than_10

0.004995412308307592

In [39]:
at_least_18 = 1 - sum((poisson_table(18,20).column(1).take(np.arange(0,18))))
at_least_18

0.7029716020753258

#### Hypergeometric

The hypergeometric distribution is similar to the binomial distribution, but where outcomes are drawn without replacement. $X$ is the number of successes out of $N$ trials, but these trials are selected out of a finite population containing $M$ objects with $n$ successes. We can think of this as an urn problem similar to those we discussed in the Lesson 8 notebook. For $N-(M-n) \leq x \leq \min(n,N)$, 
$$
f(x;N,n,M)= \frac{{n\choose k}{{M-n}\choose {N-k}}}{M\choose N}
$$

The mean and variance are 
$$
E(X)=\frac{nN}{M}
$$

and
$$
V(X)=\frac{nN(M-n)(M-N)}{M^2(M-1)}
$$

##### Example

Suppose we have an urn containing 6 blue, 9 red and 8 white balls. We select 6 at random. Let $X$ be the number of red balls we select in our sample of 6. 

1) Find the expected value of $X$.

2) Find $P(X\geq 2)$. 

3) Find the variance of $X$. Verify your calculation by generating a large random sample and computing the variance of that random sample. 

In [40]:
expected_value = (9*6)/(6+9+8)
expected_value

2.347826086956522

In [45]:
def hyper_geometric(N, M, n, x): 
    
    return (choose(n,x)* (choose((M-n),(N-x)))/ choose(M,N))

In [50]:
def geo_table(N, M, n, x):
    geo_table=ds.Table().with_columns("Number of Successes (x)",np.arange(x+1))
    geo_table=geo_table.with_columns("Probability P(X=x)", geo_table.apply(lambda y:hyper_geometric(N,M,n,y),0))
    return geo_table


In [55]:
geo_table(6,23,9,2)

Number of Successes (x),Probability P(X=x)
0,0.0297483
1,0.17849
2,0.356979


In [56]:
sum(geo_table(6,23,9,2).column(1))

0.5652173913043479

In [ ]:
prob_x_greaterthan_2 = 1-(hyper_geometric(6,23,9,0) +  hyper_geometric(6,23,9,1) +  hyper_geometric(6,23,9,2))
prob_x_greaterthan_2

In [ ]:
n = 9
N = 6
M = 23

variance_x = n*M*(M-n)*(M-N)/(M**2*(M-1))
variance_x

#### Uniform

A random variable with the discrete uniform distribution, with parameters $a$ and $b$, has an equal probability of taking on any integers in the range $[a,b)$. For $a\leq x < b$,
$$
f(x;a,b)= \frac{1}{b-a}
$$

The mean and variance are 
$$
E(X) = \frac{b+a-1}{2}
$$

and 
$$
V(X) = \frac{(b-a-1)(b-a+1)}{12}
$$


#### Multinomial

The multinomial distribution is a generalization of binomial distribution. In the binomial case, trials can result in either success or failure. In the multinomial case, there could be more than two distinct outcomes. Here, the random variable $X$ is a vector (rather than a scalar) representing the counts of each outcome. For more information regarding the multinomial distribution, please review the `scipy` documentation (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html). 

### Continuous Distributions

Now we turn to random variables that can take on any value in a range rather than only distinct points. These continous distributions differ in several ways from their discrete counterparts. 

Rather than a pmf, each continuous random variable has a **probability density function** (pdf). This too is represented by $f_X(x)$, but the function does not return a probability but rather a density. The pdf has the properties that 
1. $f(x) \geq 0$  

2. $\int_{-\infty}^{\infty} f(y) dy = 1$

Also, each random variable as a **cumulative distribution function** (cdf). This function has the same meaning as in the discrete case, but can be found via integration:  
$$ F_{X}(x) = P(X \leq x) = \int_{-\infty}^x f(y)dy $$

Suppose $Y$ is a continuous random variable that can take any value in the interval $[0,1]$. It has the pdf $f_Y(y) = 2y$ in this interval. Because $Y$ is a continuous random variable the probablility at any specific value is technically 0. We could find the probability that $Y$ takes a value in any range by integrating across this function. For example: 
$$
P(0\leq Y \leq 0.5) = \int_0^{0.5} 2y dy = y^2\bigg|_0^{0.5} = 0.25
$$

In [ ]:
from scipy import integrate

Create a function for the pdf

In [ ]:
def my_pdf(x):
    if x<0 or x>1:
        return 0
    return 2*x

In [ ]:
np.round(integrate.quad(lambda x:my_pdf(x),0,.5)[0],5)

In [ ]:
np.round(integrate.quad(lambda x:my_pdf(x),-3,2)[0],5)

### Moments

As in the discrete case, continuous random variables can be summarized using moments. However, we can use integration rather than direct sums. Let $X$ be a continuous random variable with pdf $f(x)$:
$$
E(g(x))=\int_x g(x)f(x)dx
$$

The mean, or expected value, of $X$ is found by:
$$
E(X)=\int_x xf(x)dx
$$

The variance of $X$ is:
$$
V(X)=E((X-\mu)^2)=\int_x (x-\mu)^2f(x) dx
$$

Also, the expression for variance can be simplified to $V(X)= E(X^2) - \mu^2$. 


    We can use Python to integrate numerically. To find the expected value of $X$:

In [ ]:
np.round(integrate.quad(lambda x:x*my_pdf(x),-3,2)[0],5)

##### Example

1) Let $g(x)=kx^2$ on $[0,2]$. Find $k$ that makes $g(x)$ a proper pdf. 

2) Find $P(.5<X<1.5)$
    
3) Find $E(X)$

In [ ]:
value = np.round(integrate.quad(lambda x:x**2,0,2)[0],10)
k = 1/value
k = round(k, 3)
k

In [ ]:
p_x_one_point_five = np.round(integrate.quad(lambda x:(3/8)*x**2,.5,1.5)[0],10)
p_x_one_point_five 

In [ ]:
expected_x = np.round(integrate.quad(lambda x:x *(3/8)*x**2,0,2)[0],10)
expected_x

#### Uniform

The simplest continuous distribution is the uniform distribution. Let $X$ be a random variable that has the continuous uniform distribution on $[a,b]$. $X$ has equal probability of taking any value in this range. For $a \leq x \leq b$, 
$$
f(x)=\frac{1}{b-a}
$$

The mean and variance are
$$
E(X)=\frac{a+b}{2}
$$

and 
$$
V(X)=\frac{(b-a)^2}{12}
$$

#### Exponential

The exponential distribution is closely related to the Poisson distribution. Both distributions are results of a Poisson process. Recall that in a Poisson process, if $X$ represents the number of arrivals in a certain amount of time, then $X$ follows a Poisson distribution with parameter $\lambda$, the mean number of arrivals in that amount of time. 

Let $Y$ represent the amount of time until the next arrival in this process. $Y$ follows an exponential distribution with parameter $\lambda$, the mean number of arrivals in *unit* time. Note that in `scipy`, the parameter is specified with the `scale` argument, where `scale` is equal to $1/\lambda$. 

For $y\geq 0$,
$$
f(y)=\lambda e^{-\lambda y}
$$

The mean and variance are 
$$
E(Y)=\frac{1}{\lambda}
$$
and 
$$
V(Y)=\frac{1}{\lambda^2}
$$


##### Example

Suppose fleet vehicles arrive to a maintenance garage at an average rate of 4 per day. Let's model these arrivals using the Poisson process. Let $Y$ be the time (in days) until the next arrival. 

1) What is the distribution (with parameter) of $Y$? 

2) Find the probability that no vehicles arrive in the next five days. 

3) Find the probability that the next vehicle will arrive at least 2 days from now, but before 4 days from now. 

1). distribution x ~ exp( 4)


In [ ]:
no_cars_5_days = stats.expon.sf(5,scale = 1/4)
no_cars_5_days

In [ ]:
cars_btwn_2_4 = stats.expon.cdf(4,scale = 1/4) - stats.expon.cdf(2,scale = 1/4)
cars_btwn_2_4

#### Normal

The normal (or Gaussian) distribution is an incredibly important distribution in probability and statistics. You have certainly seen a bell curve at some point and are likely aware that certain quantities take a bell shaped, or normal, distribution. 

Let $X \sim \textsf{N}(\mu,\sigma)$. The parameters $\mu$ and $\sigma$ represent the mean and standard deviation of $X$. In `scipy`, these are referred to as `loc` and `scale`. 

For $-\infty < x < \infty$,
$$
f(x;\mu,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

As mentioned, the mean and variance are $\mu$ and $\sigma^2$. 

##### Example

Let $X$ follow the normal distribution with mean 3 and standard deviation 10. 

1) Find $P(X<0)$. 

2) Find the 90th percentile of $X$. 

3) Given $X$ is greater than the mean, find the probability that $X$ is less than the mean plus two standard deviations. 

In [ ]:
p_x_less_0 = stats.norm.cdf(0,3,10)
p_x_less_0

In [ ]:
stats.norm.ppf(.9,3,10)

In [ ]:
less_than_2_sd_given_greater_than_mean = (stats.norm.cdf(23,3,10) - stats.norm.cdf(3,3,10))/(stats.norm.cdf(3,3,10))
less_than_2_sd_given_greater_than_mean

#### Gamma

The gamma distribution is a generalization of the exponential distribution. If $X$ has the gamma distribution with parameters $\alpha$ and $\lambda$. Note that in `scipy`, $\alpha$ is the shape parameter, $a$, and $\lambda$ is the inverse of the scale parameter (similar to the exponential distribution). 

For $x\geq 0$, 
$$
f(x;\alpha,\lambda)= \frac{\lambda^\alpha}{\Gamma(\alpha)}x^{\alpha-1}e^{-\lambda x}
$$

The mean and variance are:
$$
E(X)=\frac{\alpha}{\lambda}
$$

and
$$
V(X)=\frac{\alpha}{\lambda^2}
$$

##### Example

Let $X \sim \textsf{Gamma}(3,0.7)$. 

1) Find $P(X>E(X))$

2) Find $P(X\leq 1)$

In [ ]:
expected_value_x = 3/.7
expected_value_x
1 - stats.gamma.cdf(expected_value_x, 3, scale = 1/0.7)

In [ ]:
x_lessthan_eq_1 = stats.gamma.cdf(1,3, scale = 1/0.7)
x_lessthan_eq_1

In [ ]:
_,ax=plt.subplots(1,1)
x=np.arange(0.001,20,0.001)
ax.plot(x,stats.gamma.pdf(x,3,scale=1/0.7));

Note that we can plot the exponential distribution with $\lambda=0.7$ by setting $a=1$:

In [ ]:
_,ax=plt.subplots(1,1)
x=np.arange(0.001,10,0.001)
ax.plot(x,stats.gamma.pdf(x,1,scale=1/0.7));

#### Beta

The beta distribution is unique among continuous distributions in that it applies only to random variables restricted to the domain $[0,1]$. The continuous uniform distribution is a special case of the beta distribution. To learn more about the beta distribution, see the `scipy` documentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.beta.html#scipy.stats.beta

##### Example

1) What are the parameters of the beta distribution that make the distribution equivalent to $\textsf{Unif}(0,1)$? 

In [ ]:
print(stats.beta.pdf(1,1,1))
print(stats.uniform.pdf(1, 0))

beta would take the parameters x = 1, a = 1, b = 1
